Brone Layer Tables


orders_raw

In [0]:


CREATE OR REFRESH STREAMING  TABLE orders_raw
COMMENT "The raw book orders ingested from orders-raw"
AS SELECT * FROM  cloud_files("${datasets.path}/orders-raw", "parquet",
                        map("schema", "order_id STRING, order_timestamp LONG, customer_id STRING, quantity LONG"))

Customers


In [0]:


CREATE OR REFRESH MATERIALIZED VIEW customers
COMMENT "The customers lookup Table, ingested from customers-json"
AS SELECT * FROM json.`${datasets.path}/customers-json`


Silver Layer Tables

In [0]:

CREATE OR REPLACE STREAMING TABLE orders_cleaned(
  CONSTRAINT valid_order_number EXPECT(order_id IS NOT NULL) ON VIOLATION DROP ROW
)
AS 
SELECT order_id, quantity, o.customer_id, c.profile:first_name as f_name, c.profile:last_name as l_name,
cast(from_unixtime(order_timestamp, 'yyyy-MM-dd HH:mm:ss') AS timestamp) order_timestamp,
c.profile:address:country as country
FROM STREAM(LIVE.orders_raw) o
LEFT JOIN LIVE.customers c  
ON o.customer_id = c.customer_id

GOLD TABLE

In [0]:


CREATE OR REFRESH MATERIALIZED  VIEW cn_daily_customer_books
COMMENT "Daily number of books per customer in China"
AS
  SELECT customer_id, f_name, date_trunc("DD", order_timestamp) as order_date, sum(quantity) as book_counts
  FROM LIVE.orders_cleaned
  WHERE country = 'CHINA'
  GROUP BY customer_id, f_name, l_name, date_trunc("DD", order_timestamp)
  
  